In [3]:
import os

In [2]:
%pwd

'c:\\Users\\ACER\\Desktop\\Kidney_disease_predication_using_deep_learning\\research'

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\ACER\\Desktop\\Kidney_disease_predication_using_deep_learning'

In [5]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/HimanshuRajput013/Kidney_disease_predication_using_deep_learning.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="HimanshuRajput013"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e34fcb377dc3373913de7a70cd0b23417e4ced32"

In [6]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/HimanshuRajput013/Kidney_disease_predication_using_deep_learning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [14]:
import mlflow.tensorflow


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        import dagshub
        dagshub.init(repo_owner='HimanshuRajput013', repo_name='Kidney_disease_predication_using_deep_learning', mlflow=True)

        import mlflow
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            model_name = "VGG16Model"
            if tracking_url_type_store != "file":
                mlflow.tensorflow.log_model(self.model, "model", registered_model_name=model_name)
            else:
                mlflow.tensorflow.log_model(self.model, "model")


In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    #evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-06-27 18:49:06,652: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-27 18:49:06,668: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-27 18:49:06,672: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 55s 6s/step - loss: 9.5465 - accuracy: 0.5036
[2024-06-27 18:50:03,556: INFO: common: json file saved at: scores.json]


Initialized MLflow to track repo "HimanshuRajput013/Kidney_disease_predication_using_deep_learning"

[2024-06-27 18:50:08,234: INFO: helpers: Initialized MLflow to track repo "HimanshuRajput013/Kidney_disease_predication_using_deep_learning"]


Repository HimanshuRajput013/Kidney_disease_predication_using_deep_learning initialized!

[2024-06-27 18:50:08,251: INFO: helpers: Repository HimanshuRajput013/Kidney_disease_predication_using_deep_learning initialized!]


2024/06/27 18:50:10 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\ACER\AppData\Local\Temp\tmpdcnq5j8w\model\data\model\assets
[2024-06-27 18:50:13,661: INFO: builder_impl: Assets written to: C:\Users\ACER\AppData\Local\Temp\tmpdcnq5j8w\model\data\model\assets]


AttributeError: module 'tensorflow.keras' has no attribute '__version__'